## Loading data and libraries

In [9]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
!ls './Datasets ML/winery-multivariate'

_ipynb_checkpoints
wine.data.txt
wine.names.txt
winery-classification-gaussian.ipynb


In [11]:
data_1=np.loadtxt('./Animals_with_Attributes/predicate-matrix-binary.txt')
data_2=np.loadtxt('./Animals_with_Attributes/predicate-matrix-continuous.txt')
data=np.loadtxt('./Datasets ML/winery-multivariate/wine.data.txt',delimiter=',')

## Classification using Gaussian concept and EM algorithm

The classifier is quite usefull when the clusters are not spherical, i.e. they are natural occuring with variable shapes and sizes. Using gaussian distribution which takes upon the concept of probablity function the model with be quite helpfull and better than k means. This method is fast but the only con is when there are many features and insufficient data points the covariance matrix may have inf value. Thus will not work.

In [59]:
from scipy.stats import multivariate_normal

def classify(n_clus,x,mean,cov):
    prob=np.zeros((n_clus,x.shape[0]))
    for i in range(n_clus):
        multi=multivariate_normal(cov=cov[i,:,:],mean=mean[i,:],allow_singular=True)
        prob[i,:]=multi.pdf(x)
    return prob
    
def gaussian_train(x,n_clus=3,max_iter=100):
    iter=0
    done=False
    n,d=x.shape
    EM=[]
    
    mew=np.zeros(shape=(n_clus,d))
    prob=np.zeros((n_clus,n))
    pi=np.ones(n_clus)
    covar=np.zeros((n_clus,d,d))
    
    cov=np.cov(x,rowvar=False,bias=True)
    for i in range(n_clus):
        mew[i]=x[i]
        covar[i,:,]=cov
        
    while not(done):
        prob=classify(n_clus,x,mew,covar)*pi.reshape(-1,1)
        w=prob/np.sum(prob,axis=0)
        
        pi=np.sum(w,axis=1)/n
        mew=w.dot(x)/(n*pi.reshape(-1,1))
        covar=np.array([((x-mew[i]).T.dot((x-mew[i])*w[i].reshape(-1,1)))/(n*pi[i]) for i in range(n_clus)])
        
        EM.append(sum(np.log(np.sum(prob,axis=0))))
        
        if (iter>1) & (EM[iter]<EM[iter-1]):
            done=True
            
        if iter>max_iter:
            done=True
        
        iter+=1
    print('Converged at: ',iter)
    return mew,covar,pi

def train_and_test(data,n,max_iter):
    mean,cov,pi=gaussian_train(data,n_clus=n,max_iter=max_iter)
    prob=classify(n,data,mean,cov)*pi.reshape(-1,1)
    label=np.argmax(prob,axis=0)
    return label

In [60]:
label=train_and_test(x,3,max_iter=100)
np.unique(label,return_counts=1)

Converged at:  91


(array([0, 1, 2], dtype=int64), array([23, 97, 58], dtype=int64))

In [61]:
np.unique(y,return_counts=1)

(array([0., 1., 2.]), array([59, 71, 48], dtype=int64))